In [0]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Spark').getOrCreate()

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

df = spark.createDataFrame(data=data,schema=columns)

df.show(truncate=False)

+-----+------------+
Seqno|Name |
+-----+------------+
1 |john jones |
2 |tracey smith|
3 |amy sanders |
+-----+------------+

In [0]:
def convertCase(str):
    resStr=""
    arr = str.split(" ")
    for x in arr:
       resStr= resStr + x[0:1].upper() + x[1:len(x)] + " "
    return resStr 

In [0]:
""" Converting function to UDF 
StringType() is by default hence not required """
convertUDF = udf(lambda z: convertCase(z)) 

In [0]:
from pyspark.sql.functions import col
df.select(col("Seqno"), \
    convertUDF(col("Name")).alias("Name") ) \
   .show(truncate=False)


+-----+-------------+
Seqno|Name |
+-----+-------------+
1 |John Jones |
2 |Tracey Smith |
3 |Amy Sanders |
+-----+-------------+

In [0]:
def upperCase(str):
    return str.upper()

In [0]:
upperCaseUDF = udf(lambda z:upperCase(z))   

df.withColumn("Cureated Name", upperCaseUDF(col("Name"))) \
  .show(truncate=False)

+-----+------------+-------------+
Seqno|Name |Cureated Name|
+-----+------------+-------------+
1 |john jones |JOHN JONES |
2 |tracey smith|TRACEY SMITH |
3 |amy sanders |AMY SANDERS |
+-----+------------+-------------+

In [0]:
""" Using UDF on SQL """
spark.udf.register("convertUDF", convertCase)
df.createOrReplaceTempView("NAME_TABLE")
spark.sql("select Seqno, convertUDF(Name) as Name from NAME_TABLE") \
     .show(truncate=False)

+-----+-------------+
Seqno|Name |
+-----+-------------+
1 |John Jones |
2 |Tracey Smith |
3 |Amy Sanders |
+-----+-------------+

In [0]:
@udf(returnType="String") 
def upperCase(str):
    return str.upper()

df.withColumn("Cureated Name", upperCase(col("Name"))) \
.show(truncate=False)

+-----+------------+-------------+
Seqno|Name |Cureated Name|
+-----+------------+-------------+
1 |john jones |JOHN JONES |
2 |tracey smith|TRACEY SMITH |
3 |amy sanders |AMY SANDERS |
+-----+------------+-------------+

In [0]:
""""" 
No guarantee Name is not null will execute first
If convertUDF(Name) like '%John%' execute first then 
you will get runtime error
"""
spark.sql("select Seqno, convertUDF(Name) as Name from NAME_TABLE " + "where Name is not null and convertUDF(Name) like '%John%'") \
     .show(truncate=False)  

+-----+-----------+
Seqno|Name |
+-----+-----------+
1 |John Jones |
+-----+-----------+

In [0]:
""" null check """

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders"),
    ('4',None)]

df2 = spark.createDataFrame(data=data,schema=columns)
df2.show(truncate=False)
df2.createOrReplaceTempView("NAME_TABLE2")

spark.sql("select convertUDF(Name)from NAME_TABLE2") \
     .show(truncate=False)

+-----+------------+
Seqno|Name |
+-----+------------+
1 |john jones |
2 |tracey smith|
3 |amy sanders |
4 |null |
+-----+------------+

--------------------------------------------------------------------------- 
 PythonException Traceback (most recent call last)
 <command-3635773700347270> in <module> 
 11 df2 . createOrReplaceTempView ( "NAME_TABLE2" ) 
 12 
 ---> 13 spark . sql ( "select convertUDF(Name)from NAME_TABLE2" ) \ 
 14 . show ( truncate = False ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 490 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 491 else : 
 --> 492 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 
 493 
 494 def __repr__ ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 114 # Hide where the exception came from that shows a non-Pythonic 
 115 # JVM exception message. 
 --> 116 raise converted from None 
 117 else : 
 118 raise 

 PythonException : An exception was thrown from a UDF: 'AttributeError: 'NoneType' object has no attribute 'split'', from <command-3635773700347255>, line 3. Full traceback below:
Traceback (most recent call last):
 File "<command-3635773700347255>", line 3, in convertCase
AttributeError: 'NoneType' object has no attribute 'split'

In [0]:
spark.udf.register("_nullsafeUDF", lambda str: convertCase(str) if not str is None else "")

spark.sql("select _nullsafeUDF(Name) from NAME_TABLE2") \
     .show(truncate=False)

spark.sql("select Seqno, _nullsafeUDF(Name) as Name from NAME_TABLE2 " + \
          " where Name is not null and _nullsafeUDF(Name) like '%John%'") \
     .show(truncate=False) 

+------------------+
_nullsafeUDF(Name)|
+------------------+
John Jones |
Tracey Smith |
Amy Sanders |
 |
+------------------+

+-----+-----------+
Seqno|Name |
+-----+-----------+
1 |John Jones |
+-----+-----------+